# Motor Control Testing

Test notebook for the **DigitalTwinMotorDriver** and **Motor** device class.

Uses the new telescope-mcp architecture - no hardware required!

---

## Architecture Overview

```
Motor (device)          ← High-level async API
    ↓
MotorInstance           ← Runtime instance (position, status)
    ↓
MotorDriver             ← Factory for instances (DigitalTwinMotorDriver)
```

## Motor Configuration

| Axis | Range | Home | Steps/Degree |
|------|-------|------|--------------|
| ALT | 0° (zenith) → ~90° (horizon) | 0 (zenith) | ~19,167 |
| AZ | ±110,000 steps (~±50°) | 0 (center) | ~2,222 |

---
## 1. Configuration

Toggle between **Digital Twin** (simulation) and **Hardware** (real motors) mode.

In [ ]:
# ============================================
# CONFIGURATION - Choose hardware or simulation
# ============================================

# Set to True for real hardware, False for digital twin simulation
#USE_HARDWARE = False
USE_HARDWARE = True

# Hardware settings (only used when USE_HARDWARE=True)
MOTOR_BAUD_RATE = 115200

# Known Teensy hardware ID (from serial port detection)
# Format: "USB VID:PID=16C0:0483 SER=<serial_number> LOCATION=<bus-port>"
# Run the port detection cell to find your Teensy's hwid, then paste it here
TEENSY_HWID = None  # Set to string like "USB VID:PID=16C0:0483 SER=12345678 LOCATION=1-2"

print(f"✓ Mode: {'HARDWARE' if USE_HARDWARE else 'DIGITAL TWIN'}")
if TEENSY_HWID:
    print(f"✓ Teensy hwid: {TEENSY_HWID}")

In [ ]:
# ============================================
# DETECT SERIAL PORTS
# Shows available ports and identifies Teensy
# Only runs detection in hardware mode
# ============================================

import serial.tools.list_ports

teensy_port = None

if USE_HARDWARE:
    print("Scanning serial ports...\n")
    ports = list(serial.tools.list_ports.comports())

    # Look for Teensy (Teensyduino shows as "USB Serial" or contains "Teensy")
    for port in ports:
        desc = (port.description or "").lower()
        mfr = (port.manufacturer or "").lower()
        
        # Teensy typically shows as "USB Serial" with Teensyduino or PJRC
        is_teensy = (
            "teensy" in desc or 
            "teensy" in mfr or
            "pjrc" in mfr or
            (port.vid == 0x16C0 and port.pid == 0x0483)  # Teensy USB VID:PID
        )
        
        marker = " ← TEENSY" if is_teensy else ""
        print(f"  {port.device}{marker}")
        print(f"    Description: {port.description}")
        print(f"    Manufacturer: {port.manufacturer}")
        print(f"    VID:PID: {hex(port.vid)}:{hex(port.pid)}" if port.vid else "    VID:PID: N/A")
        print()
        
        if is_teensy and teensy_port is None:
            teensy_port = port.device

    print("─" * 50)
    if teensy_port:
        print(f"✓ Teensy detected: {teensy_port}")
    else:
        print("❌ No Teensy detected - hardware mode will fail!")
        print("   Connect Teensy and re-run this cell.")
else:
    print("ℹ️ Digital twin mode - skipping serial port detection")

---
## 2. Setup

Import the motor architecture from telescope-mcp.

In [ ]:
# ============================================
# SETUP - Import motor architecture
# Run this after kernel restart to pick up code changes
# ============================================

# Force reimport of modules (useful during development)
# Order matters! Reload base types first, then implementations
import importlib
import telescope_mcp.drivers.motors.types
import telescope_mcp.drivers.motors.twin
import telescope_mcp.drivers.motors.serial_controller  # Hardware driver
import telescope_mcp.drivers.motors  # Also reload the __init__.py

importlib.reload(telescope_mcp.drivers.motors.types)
importlib.reload(telescope_mcp.drivers.motors.twin)
importlib.reload(telescope_mcp.drivers.motors.serial_controller)
importlib.reload(telescope_mcp.drivers.motors)

from telescope_mcp.drivers.motors import (
    DigitalTwinMotorDriver,
    DigitalTwinMotorConfig,
    DigitalTwinMotorInstance,
    SerialMotorDriver,  # Real hardware driver
    MotorStatus,
    MotorInfo,
    MotorType,
)
from telescope_mcp.devices import Motor, MotorConfig
import asyncio

print("✓ Motor architecture imported")

In [ ]:
# ============================================
# CREATE MOTOR DRIVER
# Uses hardware or digital twin based on USE_HARDWARE setting
# ============================================

if USE_HARDWARE:
    # Real hardware - connect to Teensy motor controller
    if not teensy_port:
        raise RuntimeError(
            "No Teensy detected! Connect the motor controller and re-run the port detection cell."
        )
    
    serial_port = teensy_port
    driver = SerialMotorDriver(baudrate=MOTOR_BAUD_RATE)
    print(f"✓ SerialMotorDriver created")
    print(f"  Serial port: {serial_port}")
    print(f"  Baud rate: {MOTOR_BAUD_RATE}")
else:
    # Digital twin - simulated motors
    serial_port = None  # Not used in digital twin mode
    config = DigitalTwinMotorConfig(
        # Position limits (matching real hardware)
        altitude_min_steps=0,       # Zenith (straight up)
        altitude_max_steps=140000,  # Horizon
        azimuth_min_steps=-110000,  # CCW limit
        azimuth_max_steps=110000,   # CW limit
        
        # Home positions
        altitude_home_steps=0,      # Home at zenith
        azimuth_home_steps=0,       # Home at center
        
        # Timing simulation
        slew_speed_steps_per_sec=5000.0,
        acceleration_time_sec=0.5,
        simulate_timing=True,
    )
    driver = DigitalTwinMotorDriver(config)
    print(f"✓ DigitalTwinMotorDriver created")
    print(f"  Config: {config}")

In [ ]:
# ============================================
# OPEN MOTOR CONTROLLER
# ============================================

# Open motor controller (pass port for hardware mode)
if USE_HARDWARE:
    motor = driver.open(serial_port)
else:
    motor = driver.open()

info = motor.get_info()
print(f"✓ Motor controller opened: {info['name']}")

# Show connection settings
print(f"\n{'═' * 50}")
print("CONNECTION")
print(f"{'═' * 50}")
if USE_HARDWARE:
    print(f"  Port:      {serial_port}")
    print(f"  Baud rate: {MOTOR_BAUD_RATE}")
else:
    print(f"  Mode:      Digital Twin (simulated)")

# Show motor configuration
print(f"\n{'═' * 50}")
print("MOTOR CONFIGURATION")
print(f"{'═' * 50}")

# Import motor configs for display
from telescope_mcp.drivers.motors.serial_controller import MOTOR_CONFIGS

for motor_type in [MotorType.ALTITUDE, MotorType.AZIMUTH]:
    cfg = MOTOR_CONFIGS[motor_type]
    print(f"\n  {motor_type.value.upper()}:")
    print(f"    Range:      {cfg.min_steps:,} → {cfg.max_steps:,} steps")
    print(f"    Home:       {cfg.home_position:,} steps")
    print(f"    Steps/deg:  {cfg.steps_per_degree:,.1f}")

# Query firmware help (hardware only)
if USE_HARDWARE:
    print(f"\n{'═' * 50}")
    print("TEENSY FIRMWARE COMMANDS")
    print(f"{'═' * 50}")
    try:
        help_text = motor.get_help()
        # Indent each line for cleaner display
        for line in help_text.strip().split('\n'):
            if line.strip():
                print(f"  {line.strip()}")
    except Exception as e:
        print(f"  (Could not query firmware: {e})")

# Get initial positions
print(f"\n{'═' * 50}")
print("CURRENT POSITIONS")
print(f"{'═' * 50}")
alt_pos = motor.get_status(MotorType.ALTITUDE)
az_pos = motor.get_status(MotorType.AZIMUTH)
print(f"  ALT: {alt_pos.position_steps:,} steps")
print(f"  AZ:  {az_pos.position_steps:,} steps")

---
## 2.1 StepperAmis Raw Diagnostics

**Direct access to hardware settings via stepperamis library.**

Use this section to:
- Read ALL motor controller parameters from the Teensy
- Check velocity, acceleration, current settings
- Identify misconfiguration causing movement issues

In [ ]:
# ============================================
# STEPPER_AMIS RAW CONNECTION
# Direct connection bypassing telescope-mcp for diagnostics
# ============================================
import sys
sys.path.insert(0, '/home/mark/src/at_stepper_amis')

from stepper_amis import Stepper_amis, getAllHwids
from pprint import pprint

# Show available serial devices
print("Available serial devices:")
pprint(getAllHwids())

if USE_HARDWARE and teensy_port:
    # Close telescope-mcp connection first to avoid port conflict
    try:
        driver.close()
        print("\n✓ Closed telescope-mcp driver to release serial port")
    except:
        pass
    
    # Use known hwid if configured, otherwise search for it
    if TEENSY_HWID:
        teensy_hwid = TEENSY_HWID
        print(f"\nUsing configured Teensy hwid: {teensy_hwid}")
    else:
        # Find hwid for the Teensy port (fallback)
        from serial.tools.list_ports import comports
        teensy_hwid = None
        for cp in comports():
            if str(cp).split()[0] == teensy_port:
                teensy_hwid = cp.hwid
                break
        
        if teensy_hwid:
            print(f"\nTeensy hwid (detected): {teensy_hwid}")
            print(f"💡 TIP: Add this to TEENSY_HWID in config cell for faster connection")
        else:
            print(f"❌ Could not find hwid for {teensy_port}")
    
    if teensy_hwid:
        # Connect using hwid (Stepper_amis API)
        sa = Stepper_amis(teensy_hwid)
        print(f"✓ Stepper_amis connected via {teensy_port}")
    else:
        sa = None
else:
    print("❌ Hardware mode not enabled or no Teensy port detected")
    sa = None

In [ ]:
# ============================================
# GET ALL SETTINGS FOR ALL AXES
# Retrieves velocity, acceleration, current, position for each axis
# ============================================

print("=" * 60)
print("STEPPER AMIS FULL HARDWARE SETTINGS")
print("=" * 60)

if sa is None:
    print("❌ No connection - run the connection cell first")
else:
    for axis in range(3):
        # status() = major items (G command), status2() = minor items (g command)
        major = sa.status(axis)
        minor = sa.status2(axis)
        
        print(f"\n{'─' * 60}")
        print(f"AXIS {axis}")
        print(f"{'─' * 60}")
        
        # Key motor parameters from status (G command)
        print(f"  Position (x):     {major.get('x', 'N/A'):>10} microsteps")
        print(f"  Velocity (speed): {major.get('speed', 'N/A'):>10} μsteps/sec")
        print(f"  Accel (accel):    {major.get('accel', 'N/A'):>10} seconds")
        print(f"  Microsteps (us):  {major.get('us', 'N/A'):>10}")
        
        # Current settings
        print(f"  Current (iRun):   {major.get('iRun', 'N/A'):>10} mA")
        print(f"  Current (iHold):  {major.get('iHold', 'N/A'):>10} mA")
        
        # Limit sensor
        print(f"  Limit Enabled:    {major.get('lim', 'N/A'):>10}")
        print(f"  Limit State:      {major.get('limState', 'N/A'):>10}")
        
        # Polarity
        print(f"  Direction Pol:    {major.get('dirPol', 'N/A'):>10}")
        
        # From status2 (g command)
        print(f"  Joystick Enable:  {minor.get('jEn', 'N/A'):>10}")
        print(f"  Joystick Zero:    {minor.get('jz', 'N/A'):>10}")

    # Show raw dicts for both motor axes
    for axis in [0, 1]:
        axis_name = "ALTITUDE" if axis == 0 else "AZIMUTH"
        print(f"\n{'=' * 60}")
        print(f"RAW STATUS DICTS - AXIS {axis} ({axis_name})")
        print("=" * 60)
        print("\nstatus() [G command]:")
        pprint(sa.status(axis))
        print("\nstatus2() [g command]:")
        pprint(sa.status2(axis))

In [ ]:
# ============================================
# FIX MOTOR PARAMETERS (If needed)
# Uses Stepper_amis API: setSpeed, iMotor, iHold, setATime, usMotor
# ============================================

# ═══════════════════════════════════════════════════════════════
# AXIS MAPPING (confirmed 2026-01-03)
# ═══════════════════════════════════════════════════════════════
#   Axis 0 = ALTITUDE (ALT)
#   Axis 1 = AZIMUTH (AZ)
#   Axis 2 = (unused)
# ═══════════════════════════════════════════════════════════════

# ═══════════════════════════════════════════════════════════════
# MOTOR SPECIFICATIONS: 17HS24-2104S (NEMA 17)
# ═══════════════════════════════════════════════════════════════
# Manufacturer Part Number: 17HS24-2104S
# Motor Type:               Bipolar Stepper
# Step Angle:               1.8° (200 steps/revolution)
# Holding Torque:           65 Ncm (92 oz.in)
# Rated Current/phase:      2.1A
# Phase Resistance:         1.6Ω
# Inductance:               3mH ±20% (1KHz)
# Insulation Class:         B 130°C [266°F]
#
# MICROSTEP CALCULATION:
#   At 128 microsteps: 200 × 128 = 25,600 μsteps/revolution
# ═══════════════════════════════════════════════════════════════

# Select axis to configure
AXIS_TO_FIX = 0  # ALTITUDE motor

# Recommended settings based on 17HS24-2104S specs
# ─────────────────────────────────────────────────────────────
VELOCITY = 4000           # μsteps/sec - reasonable speed
ACCEL_TIME = 0.2          # seconds to reach velocity  
CURRENT_MOVING = 2100     # mA (100% of 2.1A rated - good torque, less heat)
CURRENT_HOLDING = 400     # mA (~25% of moving - prevents overheating)
MICROSTEPS = 128          # microstepping divisor (8, 16, 32, 64, 128)
DIR_POLARITY = 1          # 1 or -1 for direction
# ─────────────────────────────────────────────────────────────

# APPLY SETTINGS
print(f"Applying settings to Axis {AXIS_TO_FIX} (ALTITUDE)...")
print(f"  Motor: 17HS24-2104S (2.1A rated)")
print(f"  Current: {CURRENT_MOVING}mA moving / {CURRENT_HOLDING}mA hold (75% rated)")
sa.setSpeed(AXIS_TO_FIX, VELOCITY)
sa.setATime(AXIS_TO_FIX, ACCEL_TIME)
sa.iMotor(AXIS_TO_FIX, CURRENT_MOVING)
sa.iHold(AXIS_TO_FIX, CURRENT_HOLDING)
sa.usMotor(AXIS_TO_FIX, MICROSTEPS)
sa.setDirPol(AXIS_TO_FIX, DIR_POLARITY)
print("✓ Settings applied (RAM only, not saved to EEPROM)")

# Verify
print("\nVerifying new settings:")
pprint(sa.status(AXIS_TO_FIX))

In [ ]:
# ============================================
# FIX AZIMUTH MOTOR PARAMETERS (If needed)
# Uses Stepper_amis API: setSpeed, iMotor, iHold, setATime, usMotor
# ============================================

# ═══════════════════════════════════════════════════════════════
# MOTOR SPECIFICATIONS: 23HS41-1804S (NEMA 23)
# ═══════════════════════════════════════════════════════════════
# Manufacturer Part Number: 23HS41-1804S
# Motor Type:               Bipolar Stepper
# Step Angle:               1.8° (200 steps/revolution)
# Holding Torque:           ~1.26 Nm (179 oz.in)
# Rated Current/phase:      1.8A
# Phase Resistance:         1.5Ω
# Inductance:               3.6mH ±20%
#
# NOTE: Larger NEMA 23 motor for azimuth axis (horizontal rotation)
#       No gravity load = lower holding current needed
# ═══════════════════════════════════════════════════════════════

# Select axis to configure
AXIS_TO_FIX = 1  # AZIMUTH motor

# Recommended settings based on 23HS41-1804S specs
# ─────────────────────────────────────────────────────────────
VELOCITY = 4000           # μsteps/sec - slower for more torque headroom
ACCEL_TIME = 0.3          # seconds to reach velocity (larger motor = slower accel)
CURRENT_MOVING = 1800     # mA (100% of 1.8A rated - full torque)
CURRENT_HOLDING = 400       # mA (0% - horizontal axis, no gravity load)
MICROSTEPS = 128         # microstepping divisor (8, 16, 32, 64, 128)
DIR_POLARITY = 1          # 1 or -1 for direction
# ─────────────────────────────────────────────────────────────

# APPLY SETTINGS
print(f"Applying settings to Axis {AXIS_TO_FIX} (AZIMUTH)...")
print(f"  Motor: 23HS41-1804S (1.8A rated, NEMA 23)")
print(f"  Velocity: {VELOCITY} μsteps/sec (slower for more torque)")
print(f"  Current: {CURRENT_MOVING}mA moving / {CURRENT_HOLDING}mA hold")
sa.setSpeed(AXIS_TO_FIX, VELOCITY)
sa.setATime(AXIS_TO_FIX, ACCEL_TIME)
sa.iMotor(AXIS_TO_FIX, CURRENT_MOVING)
sa.iHold(AXIS_TO_FIX, CURRENT_HOLDING)
sa.usMotor(AXIS_TO_FIX, MICROSTEPS)
sa.setDirPol(AXIS_TO_FIX, DIR_POLARITY)
print("✓ Settings applied (RAM only, not saved to EEPROM)")

# Verify
print("\nVerifying new settings:")
pprint(sa.status(AXIS_TO_FIX))

In [ ]:
# ============================================
# TEST MOVE WITH STEPPER_AMIS DIRECTLY
# Bypasses telescope-mcp to test raw motor movement
# ============================================

TEST_AXIS = 1
TEST_MICROSTEPS = 64000 # ~8 full steps at 128 microsteps/step

if sa is None:
    print("❌ No connection - run the connection cell first")
else:
    print(f"Testing Axis {TEST_AXIS} with {TEST_MICROSTEPS} microsteps...")

    # Get position before
    status_before = sa.status(TEST_AXIS)
    pos_before = status_before.get('x', 0)
    print(f"  Position before: {pos_before}")

    # Get current settings for this move
    vel = status_before.get('speed', 0)
    accel = status_before.get('accel', 0)
    current = status_before.get('iRun', 0)
    print(f"  Velocity: {vel} μsteps/sec")
    print(f"  Acceleration: {accel} sec")
    print(f"  Current: {current} mA")

    # Do the move (move2 is relative move)
    print(f"\n  Moving +{TEST_MICROSTEPS} microsteps...")
    sa.move2(TEST_AXIS, TEST_MICROSTEPS)

    # Check new position
    status_after = sa.status(TEST_AXIS)
    pos_after = status_after.get('x', 0)
    delta = pos_after - pos_before

    print(f"\n  Position after: {pos_after}")
    print(f"  Actual delta: {delta} microsteps")

    if abs(delta) >= abs(TEST_MICROSTEPS) * 0.9:
        print(f"  ✓ MOVEMENT SUCCESSFUL")
    elif delta != 0:
        print(f"  ⚠️ PARTIAL MOVEMENT (expected {TEST_MICROSTEPS}, got {delta})")
    else:
        print(f"  ❌ NO MOVEMENT DETECTED")
        print(f"     Check: velocity, current, mechanical binding")

In [ ]:
# ============================================
# SAVE ALL SETTINGS TO EEPROM
# Settings will persist across power cycles
# ============================================

print("Saving motor settings to EEPROM...")
print("  ⚠️ Settings will persist across power cycles\n")

# Save ALTITUDE (Axis 0)
print("Saving ALTITUDE (Axis 0)...")
sa.storeEE(0, 1)
print("  ✓ ALTITUDE saved to EEPROM")

# Save AZIMUTH (Axis 1)
print("Saving AZIMUTH (Axis 1)...")
sa.storeEE(1, 1)
print("  ✓ AZIMUTH saved to EEPROM")

print("\n" + "=" * 50)
print("✓ ALL SETTINGS SAVED TO EEPROM")
print("=" * 50)

In [ ]:
# ============================================
# CLOSE STEPPER_AMIS CONNECTION
# Run before going back to telescope-mcp driver
# ============================================

try:
    sa.close()
    print("✓ Stepper_amis serial connection closed")
    print("  You can now re-run the telescope-mcp driver cells")
except Exception as e:
    print(f"Connection close error: {e}")
    print("  You can now re-run the telescope-mcp driver cells")
except:
    print("Connection already closed or not opened")

In [ ]:
# ============================================
# TEST ALT MOTOR - Run multiple times to verify
# ALT: +steps = DOWN toward horizon, -steps = UP toward zenith
# ============================================

TEST_STEPS = 100

# Get starting position
alt_start = motor.get_status(MotorType.ALTITUDE)
print(f"ALT starting: {alt_start.position_steps} steps")

# Move
print(f"\n--- Moving ALT +{TEST_STEPS} steps (DOWN) ---")
motor.move_relative(MotorType.ALTITUDE, TEST_STEPS)

alt_after = motor.get_status(MotorType.ALTITUDE)
print(f"ALT: {alt_start.position_steps} → {alt_after.position_steps}")
print(f"Delta: {alt_after.position_steps - alt_start.position_steps} steps")
print(f"Result: {'✓ MOVED' if alt_after.position_steps != alt_start.position_steps else '✗ NO MOVEMENT'}")

# Return to start
print(f"\n--- Returning to start ---")
motor.move(MotorType.ALTITUDE, alt_start.position_steps)
alt_final = motor.get_status(MotorType.ALTITUDE)
print(f"ALT: {alt_final.position_steps} (returned: {'✓' if alt_final.position_steps == alt_start.position_steps else '✗'})")

In [ ]:
# ============================================
# TEST AZ MOTOR - Run multiple times to verify
# AZ: +steps = CW, -steps = CCW
# ============================================

TEST_STEPS = 100

# Get starting position
az_start = motor.get_status(MotorType.AZIMUTH)
print(f"AZ starting: {az_start.position_steps} steps")

# Move
print(f"\n--- Moving AZ +{TEST_STEPS} steps (CW) ---")
motor.move_relative(MotorType.AZIMUTH, TEST_STEPS)

az_after = motor.get_status(MotorType.AZIMUTH)
print(f"AZ: {az_start.position_steps} → {az_after.position_steps}")
print(f"Delta: {az_after.position_steps - az_start.position_steps} steps")
print(f"Result: {'✓ MOVED' if az_after.position_steps != az_start.position_steps else '✗ NO MOVEMENT'}")

# Return to start
print(f"\n--- Returning to start ---")
motor.move(MotorType.AZIMUTH, az_start.position_steps)
az_final = motor.get_status(MotorType.AZIMUTH)
print(f"AZ: {az_final.position_steps} (returned: {'✓' if az_final.position_steps == az_start.position_steps else '✗'})")

---
## 3. Low-Level Driver API

Direct interaction with `DigitalTwinMotorInstance` - the protocol-compliant interface.

In [ ]:
# ============================================
# GET MOTOR CONTROLLER INSTANCE
# ============================================

# List available controllers (always 1 for digital twin)
controllers = driver.get_available_controllers()
print(f"Available controllers: {controllers}")

# Open the controller
motor = driver.open()
print(f"\n✓ Motor controller opened")
print(f"  Info: {motor.get_info()}")

In [ ]:
# ============================================
# CHECK MOTOR STATUS
# ============================================
# NOTE: Uses MotorType from setup cell - don't re-import after reload!

def show_status(motor_type: MotorType) -> None:
    """Display motor status."""
    status = motor.get_status(motor_type)
    print(f"{motor_type.value}: pos={status.position_steps:>7}, "
          f"moving={status.is_moving}, "
          f"target={status.target_steps}")

show_status(MotorType.ALTITUDE)
show_status(MotorType.AZIMUTH)

In [ ]:
# ============================================
# ABSOLUTE MOVE (Low-Level)
# Move ALT to 45° (~half way to horizon)
# ============================================
import time

target_steps = 70000  # ~45° from zenith
print(f"Moving ALT to {target_steps} steps...")

motor.move(MotorType.ALTITUDE, target_steps)

show_status(MotorType.ALTITUDE)
print("✓ Move complete")

In [ ]:
# ============================================
# RELATIVE MOVE (Low-Level)
# Nudge ALT down by 5000 steps
# ============================================

print("Nudging ALT +5000 steps...")
motor.move_relative(MotorType.ALTITUDE, 5000)

show_status(MotorType.ALTITUDE)
print("✓ Nudge complete")

In [ ]:
# ============================================
# HOME MOTOR (Low-Level)
# Return ALT to zenith
# ============================================

print("Homing ALT...")
motor.home(MotorType.ALTITUDE)

show_status(MotorType.ALTITUDE)
print("✓ ALT at home (zenith)")

---
## 3. High-Level Motor Device API

The `Motor` class provides async operations with context manager support.

In [ ]:
# ============================================
# CREATE MOTOR DEVICE
# High-level async wrapper with context manager
# ============================================

# Create a new driver for the device layer (fresh state)
device_driver = DigitalTwinMotorDriver(config)

# Create Motor device - accepts driver and optional config
motor_config = MotorConfig(
    controller_id=0,
    default_speed=100,
)

motor_device = Motor(driver=device_driver, config=motor_config)
print(f"✓ Motor device created")
print(f"  Available: {motor_device.get_available_controllers()}")

In [ ]:
# ============================================
# ASYNC MOTOR OPERATIONS
# ============================================

async def demo_async_motor():
    """Demonstrate async motor operations."""
    async with motor_device:  # Connects on enter, disconnects on exit
        print(f"Connected: {motor_device.connected}")
        print(f"Info: {motor_device.info}")
        
        # Move altitude to 30 degrees
        print("\nMoving ALT to 30° (~46666 steps)...")
        await motor_device.move_to(MotorType.ALTITUDE, steps=46666)
        status = motor_device.get_status(MotorType.ALTITUDE)
        print(f"ALT status: {status}")
        
        # Move by relative amount
        print("\nMoving ALT +5000 steps...")
        await motor_device.move_by(MotorType.ALTITUDE, steps=5000)
        status = motor_device.get_status(MotorType.ALTITUDE)
        print(f"ALT status: {status}")
        
        # Home altitude
        print("\nHoming ALT...")
        await motor_device.home(MotorType.ALTITUDE)
        status = motor_device.get_status(MotorType.ALTITUDE)
        print(f"ALT status: {status}")

await demo_async_motor()

---
## 4. Coordinated Movement

Move both axes simultaneously using async.

In [ ]:
# ============================================
# COORDINATED ALT/AZ SLEW
# ============================================

async def slew_to_altaz(alt_steps: int, az_steps: int):
    """Slew telescope to ALT/AZ position.
    
    Moves both axes concurrently for faster slewing.
    """
    # Create fresh driver and device
    slew_driver = DigitalTwinMotorDriver(config)
    slew_device = Motor(driver=slew_driver)
    
    async with slew_device:
        print(f"Slewing to ALT={alt_steps}, AZ={az_steps}")
        
        # Start both moves concurrently
        await asyncio.gather(
            slew_device.move_to(MotorType.ALTITUDE, alt_steps),
            slew_device.move_to(MotorType.AZIMUTH, az_steps),
        )
        
        alt_status = slew_device.get_status(MotorType.ALTITUDE)
        az_status = slew_device.get_status(MotorType.AZIMUTH)
        
        print(f"✓ ALT: {alt_status.position_steps} steps")
        print(f"✓ AZ:  {az_status.position_steps} steps")

# Slew to ~45° altitude, ~20° azimuth
await slew_to_altaz(70000, 16000)

In [ ]:
# ============================================
# HOME ALL AXES
# ============================================

async def home_all():
    """Return both axes to home positions."""
    home_driver = DigitalTwinMotorDriver(config)
    home_device = Motor(driver=home_driver)
    
    async with home_device:
        # First move to a non-home position
        await asyncio.gather(
            home_device.move_to(MotorType.ALTITUDE, 50000),
            home_device.move_to(MotorType.AZIMUTH, 30000),
        )
        print("Moved away from home")
        
        # Home all
        print("Homing all axes...")
        await home_device.home_all()
        
        alt_status = home_device.get_status(MotorType.ALTITUDE)
        az_status = home_device.get_status(MotorType.AZIMUTH)
        
        print(f"✓ ALT: {alt_status.position_steps} (home)")
        print(f"✓ AZ:  {az_status.position_steps} (home)")

await home_all()

---
## 5. Position Limits & Error Handling

The digital twin enforces position limits just like real hardware.

In [ ]:
# ============================================
# TEST POSITION LIMITS
# ============================================

async def test_limits():
    """Test that position limits are enforced."""
    limit_driver = DigitalTwinMotorDriver(config)
    limit_device = Motor(driver=limit_driver)
    
    async with limit_device:
        # Try to exceed altitude max (140000)
        print("Testing ALT max limit (140000)...")
        try:
            await limit_device.move_to(MotorType.ALTITUDE, 200000)
            status = limit_device.get_status(MotorType.ALTITUDE)
            print(f"  Position: {status.position_steps}")
        except ValueError as e:
            print(f"  ✓ Limit enforced: {e}")
        
        # Try to go below altitude min (0)
        print("\nTesting ALT min limit (0)...")
        try:
            await limit_device.move_to(MotorType.ALTITUDE, -5000)
            status = limit_device.get_status(MotorType.ALTITUDE)
            print(f"  Position: {status.position_steps}")
        except ValueError as e:
            print(f"  ✓ Limit enforced: {e}")
        
        # Try to exceed azimuth range
        print("\nTesting AZ limits (±110000)...")
        try:
            await limit_device.move_to(MotorType.AZIMUTH, 150000)
            status = limit_device.get_status(MotorType.AZIMUTH)
            print(f"  Position: {status.position_steps}")
        except ValueError as e:
            print(f"  ✓ Limit enforced: {e}")

await test_limits()

---
## 6. DriverFactory Integration

Use `DriverFactory` for automatic driver selection based on config.

In [ ]:
# ============================================
# DRIVER FACTORY
# Automatic driver/controller selection based on mode
# ============================================

from telescope_mcp.drivers.config import DriverFactory, DriverConfig, DriverMode

# Create factory in DIGITAL_TWIN mode
driver_config = DriverConfig(mode=DriverMode.DIGITAL_TWIN)
factory = DriverFactory(config=driver_config)

# Create motor driver through factory
motor_driver = factory.create_motor_driver()
print(f"✓ Factory mode: {factory.config.mode.value}")
print(f"✓ Driver type: {type(motor_driver).__name__}")
print(f"  Available: {motor_driver.get_available_controllers()}")

In [ ]:
# ============================================
# USE FACTORY-CREATED DRIVER WITH MOTOR DEVICE
# ============================================

async def demo_factory_motor():
    """Demonstrate Motor device with factory-created driver."""
    # Create device using factory driver
    factory_device = Motor(driver=motor_driver)
    
    async with factory_device:
        print(f"Connected via factory")
        print(f"  Type: {factory_device.info.type}")
        print(f"  Name: {factory_device.info.name}")
        
        # Quick movement test
        await factory_device.move_to(MotorType.ALTITUDE, 35000)
        status = factory_device.get_status(MotorType.ALTITUDE)
        print(f"  ALT position: {status.position_steps}")

await demo_factory_motor()

---
## 7. Fast Mode (No Timing Simulation)

Disable timing simulation for faster testing.

In [ ]:
# ============================================
# FAST MODE - No timing simulation
# ============================================

fast_config = DigitalTwinMotorConfig(
    simulate_timing=False,  # Instant moves!
)

async def demo_fast_mode():
    """Demonstrate instant moves without timing simulation."""
    fast_driver = DigitalTwinMotorDriver(fast_config)
    fast_device = Motor(driver=fast_driver)
    
    import time
    
    async with fast_device:
        start = time.time()
        
        # Move across full range - should be instant
        await fast_device.move_to(MotorType.ALTITUDE, 140000)
        await fast_device.move_to(MotorType.ALTITUDE, 0)
        await fast_device.move_to(MotorType.AZIMUTH, 110000)
        await fast_device.move_to(MotorType.AZIMUTH, -110000)
        await fast_device.move_to(MotorType.AZIMUTH, 0)
        
        elapsed = time.time() - start
        print(f"✓ 5 full-range moves completed in {elapsed:.3f}s")
        print(f"  (With timing=True, this would take ~3+ minutes)")

await demo_fast_mode()

---
## 8. Protocol Compliance Verification

Verify that implementations satisfy the defined protocols.

In [ ]:
# ============================================
# PROTOCOL COMPLIANCE
# Verify implementations satisfy protocols
# ============================================

from telescope_mcp.drivers.motors import MotorInstance, MotorDriver

# Create test instances
test_driver = DigitalTwinMotorDriver(fast_config)
test_instance = test_driver.open()

print("Protocol compliance checks:")
print(f"  MotorInstance: {isinstance(test_instance, MotorInstance)}")
print(f"  MotorDriver:   {isinstance(test_driver, MotorDriver)}")

# Verify required methods exist on instance
print("\nMotorInstance methods:")
instance_methods = ["get_info", "get_status", "move", "move_relative", "stop", "home"]
for method in instance_methods:
    has_method = hasattr(test_instance, method)
    print(f"  {method}(): {'✓' if has_method else '✗'}")

# Verify driver methods
print("\nMotorDriver methods:")
driver_methods = ["get_available_controllers", "open", "close"]
for method in driver_methods:
    has_method = hasattr(test_driver, method)
    print(f"  {method}(): {'✓' if has_method else '✗'}")

test_driver.close()

---
## 9. Azimuth Homing Sequence

Calibration procedure to find the full azimuth range of motion:

1. **Find CCW limit** - Move slowly CCW until stepper slips (stall detection)
2. **Back off** - Move CW 2-3 steps, mark as HOME (position 0)
3. **Traverse to CW limit** - Move CW until slip detected
4. **Back off** - Move CCW 2-3 steps, record total range

⚠️ **Hardware note:** Moving too hard into the end stop can damage the 3D printer belt mount. Use slow speed (20%) for limit detection.

In [ ]:
# ============================================
# HOMING SEQUENCE - Data Classes
# ============================================

from dataclasses import dataclass
from typing import Callable

@dataclass
class HomingResult:
    """Result of homing sequence.
    
    Attributes:
        home_position: Position marked as 0 (after CCW limit backoff)
        ccw_limit: Raw CCW limit position found  
        cw_limit: Raw CW limit position found
        total_range: Total steps from home to CW limit
        backoff_steps: Steps backed off from limits
    """
    home_position: int
    ccw_limit: int
    cw_limit: int  
    total_range: int
    backoff_steps: int

print("✓ HomingResult class defined")

In [ ]:
# ============================================
# RUN HOMING SEQUENCE (with simulated magnetometer)
# ============================================

# Re-import to ensure consistent types after reload
from telescope_mcp.drivers.motors.types import MotorType as MT
from telescope_mcp.drivers.motors import DigitalTwinMotorDriver, DigitalTwinMotorConfig
from telescope_mcp.devices import Motor

# Magnetometer calibration callback
magnetometer_data: list[tuple[int, float, float, float]] = []

def start_magnetometer_calibration():
    """Start recording magnetometer data during traverse."""
    print("      [Magnetometer] Calibration recording started")
    magnetometer_data.clear()

async def run_homing_demo():
    """Demonstrate the homing sequence."""
    # Use fast config for demo (no timing delays)
    homing_config = DigitalTwinMotorConfig(
        simulate_timing=False,  # Fast for demo
        azimuth_min_steps=-110000,
        azimuth_max_steps=110000,
    )
    homing_driver = DigitalTwinMotorDriver(homing_config)
    homing_device = Motor(driver=homing_driver)
    
    async with homing_device:
        # Start from a random middle position
        homing_device._instance.set_position(MT.AZIMUTH, 25000)
        print(f"Starting position: {homing_device._instance.get_status(MT.AZIMUTH).position_steps}")
        
        # Run homing sequence (use MT for consistent enum reference)
        result = await home_azimuth_internal(
            homing_device,
            motor_type=MT,
            backoff_steps=300,
            homing_speed=20,
            step_size=500,
            on_traverse_start=start_magnetometer_calibration,
        )
        
        return result

async def home_azimuth_internal(device, motor_type, backoff_steps, homing_speed, step_size, on_traverse_start):
    """Internal homing with explicit motor type reference."""
    print("=" * 50)
    print("AZIMUTH HOMING SEQUENCE")
    print("=" * 50)
    
    instance = device._instance
    
    # Step 1: Find CCW limit
    print("\n[1/4] Finding CCW limit (moving slowly)...")
    ccw_limit = instance.move_until_stall(
        motor_type.AZIMUTH, 
        direction=-1,
        speed=homing_speed,
        step_size=step_size,
    )
    print(f"      CCW limit detected at {ccw_limit} steps")
    
    # Step 2: Back off from CCW limit and mark as home
    print(f"\n[2/4] Backing off {backoff_steps} steps from CCW limit...")
    instance.move_relative(motor_type.AZIMUTH, backoff_steps, speed=50)
    home_position = instance.get_status(motor_type.AZIMUTH).position_steps
    print(f"      HOME position: {home_position} steps")
    
    # Step 3: Traverse to CW limit
    print("\n[3/4] Traversing to CW limit...")
    if on_traverse_start:
        print("      Starting magnetometer calibration...")
        on_traverse_start()
    
    cw_limit = instance.move_until_stall(
        motor_type.AZIMUTH,
        direction=+1,
        speed=homing_speed,
        step_size=step_size,
    )
    print(f"      CW limit detected at {cw_limit} steps")
    
    # Step 4: Back off from CW limit
    print(f"\n[4/4] Backing off {backoff_steps} steps from CW limit...")
    instance.move_relative(motor_type.AZIMUTH, -backoff_steps, speed=50)
    final_cw = instance.get_status(motor_type.AZIMUTH).position_steps
    
    total_range = final_cw - home_position
    
    print("\n" + "=" * 50)
    print("HOMING COMPLETE")
    print("=" * 50)
    print(f"  CCW limit:    {ccw_limit} steps")
    print(f"  HOME:         {home_position} steps") 
    print(f"  CW limit:     {cw_limit} steps")
    print(f"  Usable range: {total_range} steps")
    
    return HomingResult(
        home_position=home_position,
        ccw_limit=ccw_limit,
        cw_limit=cw_limit,
        total_range=total_range,
        backoff_steps=backoff_steps,
    )

homing_result = await run_homing_demo()

In [ ]:
# ============================================
# VERIFY HOMING RESULT
# ============================================

print("Homing Result Details:")
print(f"  home_position:  {homing_result.home_position}")
print(f"  ccw_limit:      {homing_result.ccw_limit}")
print(f"  cw_limit:       {homing_result.cw_limit}")
print(f"  total_range:    {homing_result.total_range}")
print(f"  backoff_steps:  {homing_result.backoff_steps}")

# Calculate degrees (using ~2444 steps/degree for azimuth)
steps_per_degree = 220000 / 90.0
degrees_range = homing_result.total_range / steps_per_degree
print(f"\n  Azimuth range: ~{degrees_range:.1f}°")

---
## Reference: Motor Physical Limits

### Altitude (ALT)
- **Range:** 0° (zenith) to ~90° (horizon)
- **Steps:** 0 - 140,000 (~1,556 steps/degree)
- **Home:** 0 steps (zenith)

### Azimuth (AZ)  
- **Range:** ±50° from center
- **Steps:** -110,000 to +110,000 (~2,444 steps/degree)
- **Home:** 0 steps (center)

### Architecture
```
MotorDriver (protocol)          ← Factory interface
    └── DigitalTwinMotorDriver  ← Test implementation
    └── SerialMotorDriver       ← Hardware (future)

MotorInstance (protocol)        ← Runtime interface  
    └── DigitalTwinMotorInstance

Motor (device class)            ← High-level async API
```